# Opbygning af neuralt net med numpy som udgangspunkt 

 

Casen er et 2-dimensionelt indgangsmønster.. 

 

Illustrerer sigmoid, fejlberegning ,træning o.a. grundbegreber 

 mkilde : https://realpython.com/python-ai-neural-network/
 

for nuværende indeholder notebooken kun slut resultatet.. Der er meget at gøre her!

# Første eksperiment er en neuron med 3 input og et output

Vi gør brug af noget såkaldt matrix-regning som vi gratis får med hvi vi installerer numpy.

Næste linje (starter med et udråbstegn ! ) det indikerer at linjen bliver udført i konsollen

In [1]:
!pip install --upgrade pip    

  Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\knn\pythonProject\jupytertest0\venv\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Nu går vi igang.

Sidste gang  udførte vi manuelle beregninger af et output fra en neuron.

    
Det var noget med :  X1 * W1 + X2 * W2 * X3 * W*3   (også kendt som skalarproduktet eller dot-produktet)

Afprøves herunder


In [9]:
#Først manuelt
X1 = 1
W1 =0.4
X2= 0.7
W2= 1
X3 = 0.6
W3 = 1
dotprod =  X1*W1 + X2*W2 + X3*W3
print(dotprod)

1.7000000000000002


# derefter som vektorer med numpy

In [42]:
import numpy as np

X = [1,0.7,0.6]
W = [ 0.4,1,1]

dot_product = np.dot(X,W)

print (dot_product)


1.7000000000000002


# sidste gang arbejdede vi med en "Threshold funktion"  
som Gav henholdsvis 0 eller 1 afhængig af om threshold værdien var overskredet.

In [43]:
Threshold = 1.5

def Threshold_func(input):
    global Threshold
    if (input >Threshold):
        return 1.0
    else:
        return 0.0

Threshold_func(0)

0.0

# test af threshold

In [44]:


dot_product = np.dot(X,W)

print (dot_product)


print (Threshold_func(dot_product))

1.7000000000000002
1.0


# Hvis jeg nu har en serie af data som skal behandles af mit neurale netværk..

In [37]:
# istedet for X = [1,0.7,0.6]
X = [[1,0.7,0.6],
     [1,0.1,0.8],
     [2,2.7,1.6],
     [1,1.7,0.3],
     [2,0.7,0.2]
    ]


#man kan magisk nok bruge de samme funktioner.

dot_product = np.dot(X,W)

print (dot_product)





[1.7 1.3 5.1 2.4 1.7]


# egne funktioner udvider også automatisk fra at køre på tal til at køre på vektorer.ikke

In [38]:
dot_product = np.dot(X,W)

print (dot_product)


print (Threshold_func(dot_product))

[1.7 1.3 5.1 2.4 1.7]
[11.7 11.3 15.1 12.4 11.7]


In [3]:
import numpy as np

class NeuralNetwork:
    def __init__(self, learning_rate):
        self.weights = np.array([np.random.randn(), np.random.randn()])
        self.bias = np.random.randn()
        self.learning_rate = learning_rate

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _sigmoid_deriv(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))

    def predict(self, input_vector):
        layer_1 = np.dot(input_vector, self.weights) + self.bias
        layer_2 = self._sigmoid(layer_1)
        prediction = layer_2
        return prediction

    def _compute_gradients(self, input_vector, target):
        layer_1 = np.dot(input_vector, self.weights) + self.bias
        layer_2 = self._sigmoid(layer_1)
        prediction = layer_2

        derror_dprediction = 2 * (prediction - target)
        dprediction_dlayer1 = self._sigmoid_deriv(layer_1)
        dlayer1_dbias = 1
        dlayer1_dweights = (0 * self.weights) + (1 * input_vector)

        derror_dbias = (
            derror_dprediction * dprediction_dlayer1 * dlayer1_dbias
        )
        derror_dweights = (
            derror_dprediction * dprediction_dlayer1 * dlayer1_dweights
        )

        return derror_dbias, derror_dweights

    def _update_parameters(self, derror_dbias, derror_dweights):
        self.bias = self.bias - (derror_dbias * self.learning_rate)
        self.weights = self.weights - (
            derror_dweights * self.learning_rate
        )

    def train(self, input_vectors, targets, iterations):
        cumulative_errors = []
        for current_iteration in range(iterations):
            # Pick a data instance at random
            random_data_index = np.random.randint(len(input_vectors))

            input_vector = input_vectors[random_data_index]
            target = targets[random_data_index]

            # Compute the gradients and update the weights
            derror_dbias, derror_dweights = self._compute_gradients(
                input_vector, target
            )

            self._update_parameters(derror_dbias, derror_dweights)

            # Measure the cumulative error for all the instances
            if current_iteration % 100 == 0:
                cumulative_error = 0
                # Loop through all the instances to measure the error
                for data_instance_index in range(len(input_vectors)):
                    data_point = input_vectors[data_instance_index]
                    target = targets[data_instance_index]

                    prediction = self.predict(data_point)
                    error = np.square(prediction - target)

                    cumulative_error = cumulative_error + error
                cumulative_errors.append(cumulative_error)

        return cumulative_errors


import matplotlib.pyplot as plt

input_vectors = np.array(
    [
        [3, 1.5],
        [2, 1],
        [4, 1.5],
        [3, 4],
        [3.5, 0.5],
        [2, 0.5],
        [5.5, 1],
        [1, 1],
    ]
)

targets = np.array([0, 1, 0, 1, 0, 1, 1, 0])

learning_rate = 0.1

neural_network = NeuralNetwork(learning_rate)

training_error = neural_network.train(input_vectors, targets, 10000)

print("plotting")

#plt.plot(training_error)
#plt.xlabel("Iterations")
#plt.ylabel("Error for all training instances")
#plt.savefig("cumulative_error.png")
plt.scatter(input_vectors[:,0],input_vectors[:,1],(targets+1)*8)
plt.show()
print (input_vectors)
#testing
testdata= np.array([[0,0]])
testdata = np.append(testdata,4*np.random.random(10))
testdata.reshape(2,6)
print (testdata)
testdata.reshape(6,2)
print (testdata)

#plt.scatter(testdata[:,0],testdata[:,1])   #,(targets+1)*8)
#plt.show()


ModuleNotFoundError: No module named 'matplotlib'